## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-19-13-00-00 +0000,wsj,How Much Training Should an Airline Pilot Need?,https://www.wsj.com/business/airlines/how-much...
1,2025-09-19-13-00-00 +0000,missionlocal,Sunset,https://missionlocal.org/2025/09/sf-photos-sun...
2,2025-09-19-12-59-25 +0000,bbc,Family's 'immense joy' as British couple freed...,https://www.bbc.com/news/articles/c0q7l8ewj0wo...
3,2025-09-19-12-56-13 +0000,bbc,Ministers yet to seek climate advice on Heathr...,https://www.bbc.com/news/articles/c0jq43yjx35o...
4,2025-09-19-12-52-50 +0000,nyt,What Charlie Kirk Could Mean for the Future of...,https://www.nytimes.com/2025/09/19/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2313/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
65,trump,65
44,kimmel,29
43,jimmy,24
73,new,17
22,kirk,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
155,2025-09-19-00-40-36 +0000,nypost,Jimmy Kimmel raged against Trump and his ‘MAGA...,https://nypost.com/2025/09/18/us-news/jimmy-ki...,178
314,2025-09-18-15-10-04 +0000,nypost,Trump claims Jimmy Kimmel was taken off air fo...,https://nypost.com/2025/09/18/us-news/trump-cl...,172
36,2025-09-19-11-03-25 +0000,nyt,What Jimmy Kimmel’s Suspension Says About the ...,https://www.nytimes.com/2025/09/18/business/me...,148
231,2025-09-18-20-13-00 +0000,wsj,The Trump administration asked the Supreme Cou...,https://www.wsj.com/politics/policy/trump-admi...,142
184,2025-09-18-22-16-17 +0000,nypost,Rosie O’Donnell delivers unhinged tirade about...,https://nypost.com/2025/09/18/business/rosie-o...,142


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
155,178,2025-09-19-00-40-36 +0000,nypost,Jimmy Kimmel raged against Trump and his ‘MAGA...,https://nypost.com/2025/09/18/us-news/jimmy-ki...
231,77,2025-09-18-20-13-00 +0000,wsj,The Trump administration asked the Supreme Cou...,https://www.wsj.com/politics/policy/trump-admi...
240,66,2025-09-18-19-53-00 +0000,wsj,The Trump administration is putting the weight...,https://www.wsj.com/politics/policy/jimmy-kimm...
10,47,2025-09-19-12-34-02 +0000,nyt,Europe Announces New Sanctions to Ramp Up Pres...,https://www.nytimes.com/2025/09/19/world/europ...
251,47,2025-09-18-18-59-08 +0000,bbc,Two shot dead in attack near West Bank-Jordan ...,https://www.bbc.com/news/articles/cr5qv0qmgmjo...
250,40,2025-09-18-19-04-07 +0000,bbc,First migrant deported to France under 'one in...,https://www.bbc.com/news/articles/ckg653r06jgo...
22,37,2025-09-19-11-57-25 +0000,wapo,"FTC sues Live Nation and Ticketmaster, alleges...",https://www.washingtonpost.com/business/2025/0...
313,36,2025-09-18-15-15-02 +0000,nypost,Trump admits Putin ‘let me down’ on Ukraine pe...,https://nypost.com/2025/09/18/us-news/trump-ad...
142,33,2025-09-19-01-38-44 +0000,bbc,Canada and Mexico agree to deepen ties amid Tr...,https://www.bbc.com/news/articles/cwy8r042nx2o...
177,32,2025-09-18-22-52-57 +0000,nypost,Ex-Biden chief of staff Jeff Zients called for...,https://nypost.com/2025/09/18/us-news/ex-biden...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
